# Dataset Preparation for First ICU Prediction

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import numpy as np
import pandas as pd
import spacy

from pathlib import Path

In [3]:
path = Path('./data')

## Dataset Processing

1. Load in the data
2. Drop duplicates
3. Merge `category`, `description`, and `text` into a new column called `note`
4. Tokenize text using `scispacy` and create new column called `scispacy_note` to save tokenized text
5. Save the relevant columns to a csv file onto disk

In [4]:
def tokenize_text(text):
  tokens = [token.text for token in nlp(text)]
  return ' '.join(tokens)

def group_eth(eth):
  eth = eth.lower()
  if 'white' in eth:
    return 'white'
  elif 'black' in eth:
    return 'black'
  elif 'hispanic' in eth:
    return 'hispanic'
  elif 'asian' in eth:
    return 'asian'
  else:
    return 'unknown'

In [5]:
nlp = spacy.load('en_core_sci_md', disable=['parser', 'ner', 'tagger'])

In [ ]:
df = pd.read_csv(path/'raw_dataset.csv')
df.drop_duplicates(inplace=True)
df['note'] = df['category'].str.cat(df['description'], sep='\n')
df['note'] = df['note'].str.cat(df['text'], sep='\n')
df['ethnicity'] = df['ethnicity'].apply(group_eth)
df['scispacy_note'] = df['note'].apply(tokenize_text)
df.head()

In [ ]:
df.to_csv(path/'processed_dataset.csv', index=False)

In [6]:
full_df = pd.read_csv(path/'full_raw_dataset.csv')
full_df.drop_duplicates(inplace=True)
full_df['note'] = full_df['category'].str.cat(full_df['description'], sep='\n')
full_df['note'] = full_df['note'].str.cat(full_df['text'], sep='\n')
full_df['ethnicity'] = full_df['ethnicity'].apply(group_eth)
full_df['scispacy_note'] = full_df['note'].apply(tokenize_text)
print(full_df.shape)
full_df.head()

(75273, 21)


,subject_id,hadm_id,icustay_id,admission_type,admittime,dischtime,intime,charttime,adm_icu_period,chart_icu_period,...,ethnicity,dob,gender,admission_age,category,description,text,class_label,note,scispacy_note
0,533,100009,253656,EMERGENCY,2162-05-16 15:56:00,2162-05-21 13:37:00,2162-05-17 10:18:31,2162-05-16 19:23:00,0.77,0.62,...,white,2101-07-30 00:00:00,M,60.80,Radiology,CHEST (PA & LAT),[**2162-5-16**] 7:23 PM\n CHEST (PA & LAT) ...,-1,Radiology\nCHEST (PA & LAT)\n[**2162-5-16**] 7...,Radiology \n CHEST ( PA & LAT ) \n [ * * 2162 ...
1,533,100009,253656,EMERGENCY,2162-05-16 15:56:00,2162-05-21 13:37:00,2162-05-17 10:18:31,2162-05-17 08:12:00,0.77,0.09,...,white,2101-07-30 00:00:00,M,60.80,Radiology,VEN DUP EXTEXT BIL (MAP/DVT),[**2162-5-17**] 8:12 AM\n [**Last Name (un) 12...,-1,Radiology\nVEN DUP EXTEXT BIL (MAP/DVT)\n[**21...,Radiology \n VEN DUP EXTEXT BIL ( MAP/DVT ) \n...
2,60039,100012,239289,EMERGENCY,2177-03-12 11:48:00,2177-03-22 14:30:00,2177-03-14 10:52:23,2177-03-12 16:40:00,1.96,1.76,...,white,2109-06-26 00:00:00,M,67.71,Radiology,CHEST (PRE-OP PA & LAT),[**2177-3-12**] 4:40 PM\n CHEST (PRE-OP PA & L...,1,Radiology\nCHEST (PRE-OP PA & LAT)\n[**2177-3-...,Radiology \n CHEST ( PRE-OP PA & LAT ) \n [ * ...
3,60039,100012,239289,EMERGENCY,2177-03-12 11:48:00,2177-03-22 14:30:00,2177-03-14 10:52:23,2177-03-13 13:23:00,1.96,0.90,...,white,2109-06-26 00:00:00,M,67.71,Radiology,P CAROTID SERIES COMPLETE PORT,[**2177-3-13**] 1:23 PM\n CAROTID SERIES COMPL...,-1,Radiology\nP CAROTID SERIES COMPLETE PORT\n[**...,Radiology \n P CAROTID SERIES COMPLETE PORT \n...
4,58128,100018,269533,ELECTIVE,2176-08-29 10:30:00,2176-09-06 17:13:00,2176-08-29 16:56:37,2176-08-29 15:42:00,0.27,0.05,...,white,2121-02-13 00:00:00,M,55.54,Radiology,O C-SPINE SGL 1 VIEW IN O.R.,[**2176-8-29**] 3:42 PM\n C-SPINE SGL 1 VIEW I...,-1,Radiology\nO C-SPINE SGL 1 VIEW IN O.R.\n[**21...,Radiology \n O C-SPINE SGL 1 VIEW IN O.R. \n [...


In [7]:
full_df.to_csv(path/'full_processed_dataset.csv', index=False)

## Stats

In [ ]:
df = pd.read_csv(path/'processed_dataset.csv')

In [ ]:
df.loc[(df['class_label'] == 0)].groupby('category').apply(lambda g: pd.Series(g['subject_id'].nunique()))

In [ ]:
df.loc[(df['class_label'] == 0)].groupby('category').size()